## using JAX to train the RNN

below is some code for loading the data such that you can train in any way you like

In [1]:
from mocapact.distillation import dataset
from mocapact import observables
from stable_baselines3.common.running_mean_std import RunningMeanStd
from mocapact.sb3.features_extractor import CmuHumanoidFeaturesExtractor
from torch.utils.data.dataloader import DataLoader
from mocapact.distillation.train import pad_collate

/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/ml_collections/config_flags/config_flags.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated

In [2]:
dataset_path = '../data/dataset/large/'

In [3]:
# create dataset and dataloader
train_dataset = dataset.ExpertDatasetRNN(
    hdf5_fnames=[
        f'{dataset_path}/CMU_016_15.hdf5', 
        f'{dataset_path}/CMU_016_55.hdf5'
    ],
    observables=observables.MULTI_CLIP_OBSERVABLES_SANS_ID,
    metrics_path=f'{dataset_path}/dataset_metrics.npz',
    clip_ids=None,
    min_seq_steps=1,
    max_seq_steps=1,
    n_start_rollouts=-1,
    n_rsi_rollouts=-1,
    normalize_obs=False,
    clip_len_upsampling=False,
    clip_weighted=False,
    advantage_weights=False,
    temperature=4.0,
    concat_observables=False,
    keep_hdf5s_open=False
)
train_loader = DataLoader(train_dataset, shuffle=True, pin_memory=True,
                          batch_size=8, num_workers=1, collate_fn=pad_collate)

called _create_offsets() in ExpertDatasetRNN


In [4]:
# create feature extractor
model_observables = observables.HIGH_LEVEL_OBSERVABLES
obs_rms = {}
for obs_key, obs_indices in train_dataset.observable_indices.items():
    rms = RunningMeanStd(shape=obs_indices.shape)
    rms.mean = train_dataset.proprio_mean[obs_indices]
    rms.var = train_dataset.proprio_var[obs_indices]
    rms.count = train_dataset.count
    obs_rms[obs_key] = rms
feature_extractor = CmuHumanoidFeaturesExtractor(
    observation_space=train_dataset.full_observation_space,
    observable_keys=model_observables,
    obs_rms=obs_rms
)

In [5]:
for batch_idx, batch in enumerate(train_loader):
    # observations: dictionary of (B, T, O_) tensors
    # actions: (B, T, A)
    # weights: (B, T, 1)
    # seq_len: list of B ints
    observations, actions, weights, seq_len = batch

    # preprocess observations into a feature tensor
    features = feature_extractor(observations)
    # features: (B, T, F) -> input to the RNN

    print(f'features of shape {features.shape}')
    print(f'actions of shape {actions.shape}')
    print(f'weights of shape {weights.shape}')
    print(f'seq_len: {seq_len}')

    # here goes the training code! 
    # note that features, act, weights are all torch tensors
    # if you want to use jax, you can convert them to jax arrays
    break

/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/ml_collections/config_flags/config_flags.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp
/Users/steve/Code/mocapact_conceptor/.venv_mocapact/lib/python3.10/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated

features of shape torch.Size([8, 122, 1291])
actions of shape torch.Size([8, 122, 56])
weights of shape torch.Size([8, 122])
seq_len: [122, 96, 39, 31, 36, 42, 30, 42]
